In [ ]:
!nvidia-smi
!pip install diffusers

In [ ]:
!pip install diffusers == 0.11.1
!pip install transformers scipy ftfy accelerate

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe= StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4" , troch_dtype= torch.float16)

In [ ]:
pipe = pipe.to("cuda")

In [ ]:
prompt ="a boy is riding a car "
image = pipe(prompt).images[0]

image.save(f'trial.png')

image

In [ ]:
import torch

generator = torch.Generator("cuda").manual_seed(42)
image= pipe(prompt, num_inference_steps=50, generator= generator).images[0]

image

In [ ]:
from PIL import Image

In [ ]:
def image_grid(imgs, rows, cols):
  assert len(imgs)== rows*cols

  w,h = imgs[0].size
  grid = Image.new('RGB',size=(cols*w,rows*h))
  grid_w,grid_h= grid_size


  for i, img in enumerate(imgs):
    grid.paste(img, box=(i%cols*w, i//colsh))

  return grid

num_cols=4
num_rows=4

prompt =["girls are playing in park"] *num_cols
images = pipe(prompt).images

all_images=[]
for i in range(num_rows):
  images(prompt).images
  all_images.extend(images)


grid= image_grid(all_images,rows= num_rows, cols=num_cols)

grid


In [ ]:



num_cols = 2
num_rows = 3

prompt = ["girls are playing in park"] * num_cols

all_images = []
for i in range(num_rows):
  images = pipe(prompt).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

In [ ]:
import torch
torch_device = 'cuda'if torch.cuda.is_available() else 'cpu'


In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

#Load the autoencoder model which will be used to decode the latents into image space.
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

#  The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")



In [ ]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler.from_pretrained("CompVis/stable-diffusion-v1-4",subfolder = "scheduler")

In [ ]:
#moving to GPU
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

In [ ]:
prompt=['a girl is sitting in a park']
height = 512
width = 512

num_inference_steps= 100
guidance_scale = 8
generator =torch.manual_seed(42)
batch_size=1

In [ ]:
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
  text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

In [ ]:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
  uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

In [ ]:
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

In [ ]:
latents = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents.to(torch_device)

In [ ]:
latents.shape

In [ ]:
scheduler.set_timesteps(num_inference_steps)
latents= latents*scheduler.init_noise_sigma

In [ ]:
from tqdm.auto import tqdm
from torch import autocast

for t in tqdm(scheduler.timesteps):
  # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
  latent_model_input = torch.cat([latents] * 2)

  latent_model_input = scheduler.scale_model_input(latent_model_input, t)

  # predict the noise residual
  with torch.no_grad():
    noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

  # perform guidance
  noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
  noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

  # compute the previous noisy sample x_t -> x_t-1
  latents = scheduler.step(noise_pred, t, latents).prev_sample

In [ ]:
latents = 1 / 0.18215 * latents

with torch.no_grad():
  image = vae.decode(latents).sample

In [ ]:
image = (image / 2 + 0.5).clamp(0, 1)
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0]